In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ubfc_datatape_160x36x36_train.h5"
valid_tape = "Z:/rppg/ubfc_datatape_160x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_deep_phys(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            #yield i, (j-tf.reduce_mean(j, axis=0))/(tf.math.reduce_std(j, axis=0)+1e-6)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_deep_phys(train).cache(f'{tmp}/deepphys'), to_tf_deep_phys(valid).cache(f'{tmp}/deepphys')
train, valid = to_tf_deep_phys(train), to_tf_deep_phys(valid)

In [3]:
deepphys = DeepPhys_end_to_end()
deepphys.compile(optimizer='Nadam', loss='mse')
deepphys.build(input_shape=(None, 36, 36, 3))
print(f'Flops per frame:{get_flops(deepphys, input_sig=[tf.TensorSpec([160, 36, 36, 3])])/160:.0f}')
deepphys.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:52158718
Model: "deep_phys_end_to_end"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 1)                 532899    
Total params: 532,899
Trainable params: 532,899
Non-trainable params: 0
_________________________________________________________________


In [4]:
deepphys.fit(train.shuffle(2000), validation_data=valid, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/DeepPhys_UBFC.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
deepphys.load_weights('../weights/DeepPhys_UBFC.h5')

Epoch 1/10
485/485 [==============================] - 29s 44ms/step - loss: 0.2515 - val_loss: 0.2203

Epoch 00001: val_loss improved from inf to 0.22034, saving model to ../weights\DeepPhys_UBFC.h5
Epoch 2/10
485/485 [==============================] - 23s 41ms/step - loss: 0.2518 - val_loss: 0.2126

Epoch 00002: val_loss improved from 0.22034 to 0.21255, saving model to ../weights\DeepPhys_UBFC.h5
Epoch 3/10
485/485 [==============================] - 21s 37ms/step - loss: 0.2492 - val_loss: 0.2160

Epoch 00003: val_loss did not improve from 0.21255
Epoch 4/10
485/485 [==============================] - 21s 36ms/step - loss: 0.2476 - val_loss: 0.2177

Epoch 00004: val_loss did not improve from 0.21255
Epoch 5/10
485/485 [==============================] - 21s 36ms/step - loss: 0.2470 - val_loss: 0.2133

Epoch 00005: val_loss did not improve from 0.21255
Epoch 6/10
485/485 [==============================] - 24s 42ms/step - loss: 0.2461 - val_loss: 0.2143

Epoch 00006: val_loss did not imp

In [5]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [6]:
eval_on_dataset(test_set_UBFC_rPPG2, deepphys, 160, (36, 36), step=1, batch=0, save='../results/DeepPhys_UBFC_PURE.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/DeepPhys_UBFC_PURE.h5')
get_metrics('../results/DeepPhys_UBFC_PURE.h5')

100%|██████████| 42/42 [00:20<00:00,  2.01it/s]


{'Sliding window': {'MAE': 11.298, 'RMSE': 20.214, 'R': 0.48887},
 'Whole video': {'MAE': 10.036, 'RMSE': 17.863, 'R': 0.62736}}